In [1]:
# ----------------------------------
# Neural Network SMS Text Classifier
# ----------------------------------

# import libraries
%pip install tf-nightly
%pip install tensorflow-datasets

import tensorflow as tf
import pandas as pd
from tensorflow import keras
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.0.1 -> 25.1.1
[notice] To update, run: C:\Users\Mofey\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.0.1 -> 25.1.1
[notice] To update, run: C:\Users\Mofey\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


2.20.0-dev20250516


In [2]:
# get data files

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

In [3]:
# Load the TSV data into DataFrames
train_df = pd.read_csv('train-data.tsv', sep='\t', header=None, names=['label', 'text'])
valid_df = pd.read_csv('valid-data.tsv', sep='\t', header=None, names=['label', 'text'])

# Preprocess the data: convert 'ham' to 0 and 'spam' to 1, and return text and labels
def preprocess(df):
    # Make a copy of the original DataFrame
    df_copy = df.copy()

    # Convert 'ham' to 0 and 'spam' to 1 in the 'label' column
    df_copy['label'] = df_copy['label'].map({'ham': 0, 'spam': 1})
    
    # Change the label column to float32 type
    df_copy['label'] = df_copy['label'].astype(np.float32)

    # Return the 'text' and 'label' columns as arrays
    texts = df_copy['text'].values
    labels = df_copy['label'].values

    return texts, labels

train_texts, train_labels = preprocess(train_df)
valid_texts, valid_labels = preprocess(valid_df)

# Build a text vectorizer layer and adapt it to the training texts
vectorizer = keras.layers.TextVectorization()
vectorizer.adapt(train_texts)

# Create the RNN model
vocab_size = len(vectorizer.get_vocabulary())
model = keras.Sequential([
    vectorizer,
    keras.layers.Embedding(vocab_size, 64, mask_zero=True),
    keras.layers.Bidirectional(keras.layers.LSTM(64)),
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dense(1, activation='sigmoid')
])

model.compile(
    loss='binary_crossentropy',
    optimizer=keras.optimizers.Adam(1e-4),
    metrics=['accuracy']
)

model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ text_vectorization              │ ?                      │   0 (unbuilt) │
│ (TextVectorization)             │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ embedding (Embedding)           │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional (Bidirectional)   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [4]:
# Stopping condition
stop_condition = tf.keras.callbacks.EarlyStopping(monitor='val_loss',
                                  mode="min",
                                  patience=3,
                                  restore_best_weights=True)

history = model.fit(x=train_texts,y=train_labels, epochs=15,
                    validation_data=(valid_texts,valid_labels),
                    callbacks=[stop_condition]
                    )

Epoch 1/15
131/131 ━━━━━━━━━━━━━━━━━━━━ 18s 70ms/step - accuracy: 0.8428 - loss: 0.6561 - val_accuracy: 0.8657 - val_loss: 0.4385
Epoch 2/15
131/131 ━━━━━━━━━━━━━━━━━━━━ 8s 59ms/step - accuracy: 0.8703 - loss: 0.3712 - val_accuracy: 0.9361 - val_loss: 0.2022
Epoch 3/15
131/131 ━━━━━━━━━━━━━━━━━━━━ 7s 55ms/step - accuracy: 0.9560 - loss: 0.1754 - val_accuracy: 0.9784 - val_loss: 0.1091
Epoch 4/15
131/131 ━━━━━━━━━━━━━━━━━━━━ 7s 54ms/step - accuracy: 0.9871 - loss: 0.0901 - val_accuracy: 0.9828 - val_loss: 0.0681
Epoch 5/15
131/131 ━━━━━━━━━━━━━━━━━━━━ 8s 58ms/step - accuracy: 0.9928 - loss: 0.0450 - val_accuracy: 0.9835 - val_loss: 0.0563
Epoch 6/15
131/131 ━━━━━━━━━━━━━━━━━━━━ 7s 52ms/step - accuracy: 0.9970 - loss: 0.0237 - val_accuracy: 0.9828 - val_loss: 0.0554
Epoch 7/15
131/131 ━━━━━━━━━━━━━━━━━━━━ 7s 50ms/step - accuracy: 0.9961 - loss: 0.0244 - val_accuracy: 0.9856 - val_loss: 0.0479
Epoch 8/15
131/131 ━━━━━━━━━━━━━━━━━━━━ 7s 51ms/step - accuracy: 0.9973 - loss: 0.0171 - val_acc

In [5]:
# function to predict messages based on model
# (should return list containing prediction and label, ex. [0.008318834938108921, 'ham'])
def predict_message(text):
    # Convert the input text into a TensorFlow tensor of strings
    input_tensor = tf.convert_to_tensor([text])
    # Get the prediction probability from the model
    probability = model.predict(input_tensor)[0][0]
    # Determine the label based on the probability threshold
    label = 'spam' if probability > 0.5 else 'ham'
    return probability, label

# Test the function
pred_text = "how are you doing today?"
print(predict_message(pred_text))

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 971ms/step
(np.float32(0.0011078406), 'ham')


In [6]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 84ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 84ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step
You passed the challenge. Great job!
